In [1]:
import numpy as np
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import confusion_matrix
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import Ridge
from sklearn.model_selection import learning_curve
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import PolynomialFeatures
from sklearn.preprocessing import OrdinalEncoder, Binarizer, LabelBinarizer, KBinsDiscretizer
from sklearn.preprocessing import RobustScaler
from sklearn.linear_model import Lasso
from scipy.stats import pearsonr
import pandas as pd
from sklearn.datasets import make_classification, load_digits
from sklearn.compose import ColumnTransformer
from sklearn.compose import make_column_transformer
from sklearn.feature_selection import chi2, SelectKBest

from scipy.stats import f_oneway
import pandas as pd 
import matplotlib.pyplot as plt
from sklearn.linear_model import ElasticNet
from sklearn.pipeline import make_pipeline
from sklearn.pipeline import Pipeline



In [2]:
data_cleaned = pd.read_csv('data_cleaned.csv')
data_cleaned

,age,sex,bmi,children,smoker,region,charges,region_northeast,region_northwest,region_southeast,region_southwest,categorie_imc
0,19,0,27.900,0,1,southwest,16884.92400,0,0,0,1,surpoids
1,18,1,33.770,1,0,southeast,1725.55230,0,0,1,0,obésité modérée
2,28,1,33.000,3,0,southeast,4449.46200,0,0,1,0,obésité modérée
3,33,1,22.705,0,0,northwest,21984.47061,0,1,0,0,poids normal
4,32,1,28.880,0,0,northwest,3866.85520,0,1,0,0,surpoids
...,...,...,...,...,...,...,...,...,...,...,...,...
1331,50,1,30.970,3,0,northwest,10600.54830,0,1,0,0,obésité modérée
1332,18,0,31.920,0,0,northeast,2205.98080,1,0,0,0,obésité modérée
1333,18,0,36.850,0,0,southeast,1629.83350,0,0,1,0,obésité sévère
1334,21,0,25.800,0,0,southwest,2007.94500,0,0,0,1,surpoids


In [3]:
X = data_cleaned[["age", "sex", "children", "smoker","region", "categorie_imc"]]
y = data_cleaned["charges"]

In [8]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.15, shuffle=True, random_state=42, stratify=X["smoker"])

In [32]:
preprocessor = make_pipeline(make_column_transformer((StandardScaler(), ['children','age']),
                                                     (OrdinalEncoder(), ['smoker', 'sex']), (OneHotEncoder(),['region',"categorie_imc"])), PolynomialFeatures(2))

In [14]:
preprocessor.fit_transform(X_train)

array([[ 1.        ,  2.42210801,  0.78291086, ...,  0.        ,
         0.        ,  0.        ],
       [ 1.        ,  0.75649602,  0.42741245, ...,  0.        ,
         0.        ,  0.        ],
       [ 1.        , -0.90911597,  1.28060863, ...,  0.        ,
         0.        ,  0.        ],
       ...,
       [ 1.        , -0.07630998, -0.35468405, ...,  0.        ,
         0.        ,  1.        ],
       [ 1.        , -0.90911597,  1.35170831, ...,  0.        ,
         0.        ,  0.        ],
       [ 1.        , -0.07630998, -1.35007959, ...,  0.        ,
         0.        ,  0.        ]], shape=(1135, 120))

In [33]:
pipeline = Pipeline(steps=[
    ("preprocessor", preprocessor),
    ("polynomial_features", PolynomialFeatures()),
    ("linear_regression", LinearRegression())
])

param_grid = {
    "polynomial_features__degree":[1, 2, 3]
}

grid = GridSearchCV(pipeline, param_grid, cv=5, scoring="r2")

grid.fit(X_train, y_train)

print(grid.score(X_test, y_test))
print(grid.best_params_)

KeyboardInterrupt: 

Modele Lasso

In [19]:
X = data_cleaned.drop(["bmi", "charges"], axis=1)
y = data_cleaned["charges"]

In [25]:
preprocessor = make_pipeline(make_column_transformer((StandardScaler(), ['children','age']),
                                                     (OrdinalEncoder(), ['smoker', 'sex']), (OneHotEncoder(),['region',"categorie_imc"])), PolynomialFeatures(2))

In [28]:
pipeline = Pipeline([
    ("preprocessor", preprocessor),
    ("lasso", Lasso(alpha=0.1))
])

param_grid = {
    "lasso__alpha":[0.1,1,10,20,50,100]
}

grid_search = GridSearchCV(pipeline, param_grid, cv=5, scoring="r2")

grid_search.fit(X_train, y_train)






y_pred = grid_search.predict(X_test)


print("best parameter:", grid_search.best_params_)

print("Modelscore:", grid_search.score(X_test, y_test))

/home/flavigny/Documents/DEV_IA/assurance/Projet_assurance/.venv/lib/python3.12/site-packages/sklearn/linear_model/_coordinate_descent.py:695: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 2.090e+09, tolerance: 1.299e+07
  model = cd_fast.enet_coordinate_descent(
/home/flavigny/Documents/DEV_IA/assurance/Projet_assurance/.venv/lib/python3.12/site-packages/sklearn/linear_model/_coordinate_descent.py:695: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 2.911e+09, tolerance: 1.305e+07
  model = cd_fast.enet_coordinate_descent(
/home/flavigny/Documents/DEV_IA/assurance/Projet_assurance/.venv/lib/python3.12/site-packages/sklearn/linear_model/_coordinate_descent.py:695: ConvergenceWarning: Objective did not converge. You

best parameter: {'lasso__alpha': 20}
Modelscore: 0.8723144374326014


### modele Ridge

In [30]:
pipeline = Pipeline([
    ("preprocessor", preprocessor),
    ("ridge", Ridge())
])

param_grid = {
    "ridge__alpha":[0.1,1,10,20,50,100]
}

grid_search = GridSearchCV(pipeline, param_grid, cv=5, scoring="r2")

grid_search.fit(X_train, y_train)






y_pred = grid_search.predict(X_test)


print("best parameter:", grid_search.best_params_)

print("Modelscore:", grid_search.score(X_test, y_test))

best parameter: {'ridge__alpha': 1}
Modelscore: 0.8709635562534417


###elasticNet

In [31]:
pipeline = Pipeline([
    ("preprocessor", preprocessor),
    ("elasticnet", ElasticNet())
])

param_grid = {
    "elasticnet__alpha":[0.1,1,10,20,50,100],
    "elasticnet__l1_ratio":[0.1, 0.5, 0.7, 1]

}

grid_search = GridSearchCV(pipeline, param_grid, cv=5, scoring="r2")

grid_search.fit(X_train, y_train)






y_pred = grid_search.predict(X_test)


print("best parameter:", grid_search.best_params_)

print("Modelscore:", grid_search.score(X_test, y_test))

/home/flavigny/Documents/DEV_IA/assurance/Projet_assurance/.venv/lib/python3.12/site-packages/sklearn/linear_model/_coordinate_descent.py:695: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 2.090e+09, tolerance: 1.299e+07
  model = cd_fast.enet_coordinate_descent(
/home/flavigny/Documents/DEV_IA/assurance/Projet_assurance/.venv/lib/python3.12/site-packages/sklearn/linear_model/_coordinate_descent.py:695: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 2.911e+09, tolerance: 1.305e+07
  model = cd_fast.enet_coordinate_descent(
/home/flavigny/Documents/DEV_IA/assurance/Projet_assurance/.venv/lib/python3.12/site-packages/sklearn/linear_model/_coordinate_descent.py:695: ConvergenceWarning: Objective did not converge. You

best parameter: {'elasticnet__alpha': 20, 'elasticnet__l1_ratio': 1}
Modelscore: 0.8723144374326014
